In [2]:
!pip install tqdm


In [7]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from pathlib import Path
from tqdm import tqdm

# === CONFIG ===
RGB_PATH =r"C:\Users\khuza\OneDrive\Desktop\Data science course\DATA SCIENCE COURSE\PROJECTS\AI-DRIVEN SATELLITE ANALYSIS\data\raw\bangalore_rgb.tif"
MASK_PATH = "data/raw/bangalore_mask.tif"
PATCH_SIZE = 128
STRIDE = 128
IMAGE_SAVE_DIR = "data/patches/images"
MASK_SAVE_DIR = "data/patches/masks"
os.makedirs(IMAGE_SAVE_DIR, exist_ok=True)
os.makedirs(MASK_SAVE_DIR, exist_ok=True)

# === LOAD TIF IMAGES ===
def load_image(path, is_mask=False):
    with rasterio.open(path) as src:
        data = src.read()
    return data[0] if is_mask else data.transpose(1, 2, 0)

# === NORMALIZE RGB TO [0, 1] ===
def normalize(image):
    return image / 10000.0

# === RESIZE MASK TO MATCH RGB IF SHAPE DIFFERS ===
def resize_if_needed(mask, target_shape):
    if mask.shape != target_shape[:2]:
        print("⚠️ Mask shape doesn't match RGB. Resizing...")
        from skimage.transform import resize
        return resize(mask, target_shape[:2], order=0, preserve_range=True).astype(mask.dtype)
    return mask

# === EXTRACT PATCHES ===
def extract_image_mask_patches(image, mask, patch_size, stride):
    H, W = image.shape[:2]
    patch_id = 0
    for i in tqdm(range(0, H - patch_size + 1, stride), desc="Extracting patches"):
        for j in range(0, W - patch_size + 1, stride):
            img_patch = image[i:i+patch_size, j:j+patch_size, :]
            mask_patch = mask[i:i+patch_size, j:j+patch_size]

            np.save(f"{IMAGE_SAVE_DIR}/patch_{patch_id:04d}.npy", img_patch)
            np.save(f"{MASK_SAVE_DIR}/patch_{patch_id:04d}.npy", mask_patch)

            patch_id += 1
    print(f"✅ Extracted {patch_id} (image, mask) patch pairs.")

# === MAIN PIPELINE ===
if __name__ == "__main__":
    image = load_image(RGB_PATH, is_mask=False)
    mask = load_image(MASK_PATH, is_mask=True)

    mask = resize_if_needed(mask, image.shape)
    image = normalize(image)

    extract_image_mask_patches(image, mask, PATCH_SIZE, STRIDE)

RasterioIOError: data/raw/bangalore_mask.tif: No such file or directory